<h1> Creating the DataFrame </h1>


This notebook reads the raw txt files from Ash, which contains text content of first edition of Encyclopaedia Britannica  and converts it to a pandas data frame.



In [333]:
import glob
import re
import regex
import json
import pandas as pd
import os

<h5>  Dataframe Columns </h5>

MMSID<br>
editionTitle<br>
editor <br>
editor_date <br>
genre<br>
language<br>
termsOfAddress<br>
physicalDescription<br>
place<br>
publisher<br>
referencedBy<br>
shelfLocator<br>
editionSubTitle<br>
volumeTitle<br>
year<br>
volumeId<br>
permanentURL<br>
publisherPersons<br>
volumeNum<br>
letters<br>
part0<br>
editionNum<br>
numberOfVolumes<br>
term<br>
definition<br>
header<br>
startsAt<br>
endsAt<br>
numberOfTerms<br>
numberOfWords<br>
position<br>
termType<br>
filePath

In [334]:
#---------------------hardcoded columns to match dataframe structure---------------#
"""
edition_volumes = [
    [{"MMSID": "992277653804341", "editor": "Smellie, William", "editionTitle": "Edition 1, 1771", "editor_date": "1740-1795", "genre": "encyclopedia", "editionNum": 1, "language": "eng", "termsOfAddress": "Sir","physicalDescription": "3 v., 160 plates : ill. ; 26 cm. (4to)", "editionSubTitle": "Illustrated with one hundred and sixty copperplates", "place": "Edinburgh", "publisher": "A. & C. Black", "referencedBy": "0.0", "shelfLocator": "EB.1", "publisherPersons": [], "year": 1842, "part": None, "numberOfVolumes": 3, "volumeTitle": "Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan", "numberOfPages": None, "letters": "A-B", "volumeNum": 1, "volumeId": "144133901"},
     {"MMSID": "992277653804341", "editor": "Smellie, William", "editionTitle": "Edition 1, 1771", "editor_date": "1740-1795", "genre": "encyclopedia", "editionNum": 1, "language": "eng", "termsOfAddress": "Sir","physicalDescription": "3 v., 160 plates : ill. ; 26 cm. (4to)", "editionSubTitle": "Illustrated with one hundred and sixty copperplates", "place": "Edinburgh", "publisher": "A. & C. Black", "referencedBy": "0.0", "shelfLocator": "EB.1", "publisherPersons": [], "year": 1842, "part": None, "numberOfVolumes": 3, "volumeTitle": "Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan", "numberOfPages": None, "letters": "C-L", "volumeNum": 2, "volumeId": "144133902"},
     {"MMSID": "992277653804341", "editor": "Smellie, William", "editionTitle": "Edition 1, 1771", "editor_date": "1740-1795", "genre": "encyclopedia", "editionNum": 1, "language": "eng", "termsOfAddress": "Sir","physicalDescription": "3 v., 160 plates : ill. ; 26 cm. (4to)", "editionSubTitle": "Illustrated with one hundred and sixty copperplates", "place": "Edinburgh", "publisher": "A. & C. Black", "referencedBy": "0.0", "shelfLocator": "EB.1", "publisherPersons": [], "year": 1842, "part": None, "numberOfVolumes": 3, "volumeTitle": "Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan", "numberOfPages": None, "letters": "M-Z", "volumeNum": 3, "volumeId": "144133903"},
    ]
]

"""
#----------------------------------------------------------------------------#

'\nedition_volumes = [\n    [{"MMSID": "992277653804341", "editor": "Smellie, William", "editionTitle": "Edition 1, 1771", "editor_date": "1740-1795", "genre": "encyclopedia", "editionNum": 1, "language": "eng", "termsOfAddress": "Sir","physicalDescription": "3 v., 160 plates : ill. ; 26 cm. (4to)", "editionSubTitle": "Illustrated with one hundred and sixty copperplates", "place": "Edinburgh", "publisher": "A. & C. Black", "referencedBy": "0.0", "shelfLocator": "EB.1", "publisherPersons": [], "year": 1842, "part": None, "numberOfVolumes": 3, "volumeTitle": "Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan", "numberOfPages": None, "letters": "A-B", "volumeNum": 1, "volumeId": "144133901"},\n     {"MMSID": "992277653804341", "editor": "Smellie, William", "editionTitle": "Edition 1, 1771", "editor_date": "1740-1795", "genre": "encyclopedia", "editionNum": 1, "language": "eng", "termsOfAddress": "Sir","physicalDescription": "3 v., 160 plates : ill. 

In [335]:
def parse_term_name_without_parenthesis(term_name):
    last_or_index = term_name.rfind(" or")
    if last_or_index == -1:
        last_or_index = term_name.rfind(" Or")
    name = term_name.upper()
    alter_names = []
    if last_or_index > -1:
            text_before_or = term_name[:last_or_index]
            name_after_or = term_name[last_or_index + 4:]
            if text_before_or[-1] == ",":
                # all names after comma in text_before_or are alternative names
                # include the one before " or"
                indexes_of_comma = [index for index, char in enumerate(text_before_or) if char == ',']
                name = text_before_or[:indexes_of_comma[0]].upper()
                if len(indexes_of_comma) > 1:
                    for i in range(len(indexes_of_comma) - 1):
                        alter_name_after_comma = text_before_or[indexes_of_comma[i]+2:indexes_of_comma[i+1]]
                        alter_names.append(alter_name_after_comma.upper())
                alter_names.append(name_after_or.upper())
            else:
                # name after "or" is only alternative name for last word in text_before_or
                name = text_before_or.upper()
                index_of_last_white_space = text_before_or.rfind(" ")
                if index_of_last_white_space > -1:
                    text_before_last_word_in_text_before_or = text_before_or[:index_of_last_white_space + 1]
                    alter_names.append((text_before_last_word_in_text_before_or + name_after_or).upper())
                else:
                    alter_names.append(name_after_or.upper())
    else:
        # name = term_name
        pass
    return name, alter_names

In [336]:
def parse_term_name(term_name):
    """
    This function extract note about this term name, and term name, and alternative names.
    :param term_name: string with names of a term. e.g. ABA (or rather ABAU) HANIFA or HANFA
    :return: note: string (None if no note), name: name string (primary name), alter_names: list of alternative names.
    """
    parenthesis_pattern = regex.compile(r'\s\([\p{L}\p{N}\s\,\.]+\)')
    parenthesis_match = parenthesis_pattern.search(term_name)
    name = term_name.upper()
    note = None
    alter_names = []
    if parenthesis_match:
        text_without_parenthesis = term_name[:parenthesis_match.start()] + term_name[parenthesis_match.end():]
        name, alter_names = parse_term_name_without_parenthesis(text_without_parenthesis)

        parenthesis_match_text = parenthesis_match.group()

        # extract note, or alternative name in parentheses
        or_in_parenthesis_index = parenthesis_match_text.find("(or")
        if or_in_parenthesis_index > -1:
            first_upper_char_index = regex.search("\p{Lu}", parenthesis_match_text).start()
            alter_name_in_parenthesis = parenthesis_match_text[first_upper_char_index:-1]
            alter_names.append((alter_name_in_parenthesis + name[parenthesis_match.start():]).upper())
        else:
            note = parenthesis_match_text[2:-1]

    else:
        name, alter_names = parse_term_name_without_parenthesis(term_name)

    return note, name, alter_names

In [337]:
# test parse_term_name function
parse_term_name__tests = ["ANAXIMENES", "ABACK (a sea term)", "AGARIC Mineral", "ARMSTRONG, John, M.D.", "ARCHEUS (from the principal, chief, or first mover)", "ARUNDELIAN Marbles, Oxford Marbles, or Parian", "ARAUSIO, Civitas Arausiensis or Arausicorum", "ANTIOCHIAN Sect or Academy", "ABA (or rather ABAU) HANIFA or HANFA", "ASPHALTITES, or Lake of Bitumen", "ANTIPAS Herod, or Herod"]
for test_text in parse_term_name__tests:
    print(parse_term_name(test_text))

(None, 'ANAXIMENES', [])
('a sea term', 'ABACK', [])
(None, 'AGARIC MINERAL', [])
(None, 'ARMSTRONG, JOHN, M.D.', [])
('from the principal, chief, or first mover', 'ARCHEUS', [])
(None, 'ARUNDELIAN MARBLES', ['OXFORD MARBLES', 'PARIAN'])
(None, 'ARAUSIO, CIVITAS ARAUSIENSIS', ['ARAUSIO, CIVITAS ARAUSICORUM'])
(None, 'ANTIOCHIAN SECT', ['ANTIOCHIAN ACADEMY'])
(None, 'ABA HANIFA', ['ABA HANFA', 'ABAU HANIFA'])
(None, 'ASPHALTITES', ['LAKE OF BITUMEN'])
(None, 'ANTIPAS HEROD', ['HEROD'])


In [338]:
def find_overlap_count(string_a, string_b):
    """
    Find overlap part between the tail of string_a and head of string_b
    :param string_a:
    :param string_b:
    :return: the number of overlap characters
    """
    max_overlap = min(len(string_a), len(string_b))
    overlap_count = 0

    for i in range(1, max_overlap + 1):
        if string_a[-i:] == string_b[:i]:
            overlap_count = i

    return overlap_count

In [339]:
def extract_reference_term_name(description):
    """
    This function extracts reference terms after the word "See". If there are same reference name occurs multiple times, this function will only return one.
    :param description: description of a term
    :return: a list of reference term names
    """
    # Find all matches and their end indices using re.finditer
    matches = [(match.end()) for match in re.finditer(" (?i:see)[\,\. ]", description)]
    reference_term_names = set()
    for i in range(len(matches)):
        part_after_match = ""
        if i == len(matches) - 1:
            part_after_match = description[matches[i]:]
        else:
            part_after_match = description[matches[i]:matches[i+1]]
        part_after_match = part_after_match.strip()
        word_pattern_str = "\p{Lu}[\p{Lu}\-\']*"
        reference_term_pattern_str = word_pattern_str +  "(,?\s"+ word_pattern_str +")*(\s(and)\s" + word_pattern_str + ")?(?:[\.\,]|$)"
        reference_term_name_pattern = regex.compile(r"^" + reference_term_pattern_str)
        reference_term_name_match = reference_term_name_pattern.match(part_after_match)
        if reference_term_name_match:
            reference_term_name = reference_term_name_match.group()
            if not reference_term_name[-1].isupper():
                reference_term_name = reference_term_name[:-1]
            and_index = reference_term_name.find(" and ")
            if and_index > -1:
                reference_term_names.add(reference_term_name[:and_index])
                reference_term_names.add(reference_term_name[and_index+5:])
            else:
                # Does not support Table, figure, plate
                if reference_term_name.find("TABLE") > -1:
                    continue
                if reference_term_name.find("PLATE") > -1:
                    continue
                if reference_term_name.find("FIGURE") > -1:
                    continue
                reference_term_names.add(reference_term_name)
    return list(reference_term_names)


In [340]:
# test function extract_reference_term_name
extract_reference_term_name_test_texts = [
    "by the same name. See plate I. fig. I. ",
    "Fructification. See Plate LIV. FIG. 1. Spatha, a species of calix openin",
    "which is nou termed diarthrosis. See ANATOMY,",
    "which is nou termed diarthrosis. See ANATOMY. Something else, SEE ANATOMY ",
    "in surgery. See ABDUCTION",
    " or lambs. See MILK, RUNNET.",
    " in gardening, signifies grafting by approach. See GRAPTING and GARDENINCIS, of",
    "otherwise they despise it. See CHEMISTRY, Of fermentation.",
    "being put, will be set on fire. See OPTICS. BURNING-mountains. See VOLCANO."
]

for test_text in extract_reference_term_name_test_texts:
    print(extract_reference_term_name(test_text))

[]
[]
['ANATOMY']
['ANATOMY']
['ABDUCTION']
['MILK, RUNNET']
['GRAPTING', 'GARDENINCIS']
['CHEMISTRY']
['OPTICS', 'VOLCANO']


In [341]:
def create_term_info(end_page_num, start_page_num,  term_name, description, position, header):
    note, name, alter_names = parse_term_name(term_name)
    # remove the VOL. info from description
    description = re.sub(r"VOL\. [I\d]\. [Nn][Oo]\. [I\d]\.", "", description)
    number_of_words = len(description.split())
    reference_terms = extract_reference_term_name(description)

    term_type = "Article"
    # Topic can only have one word name
    if (len(name.split())) == 1 and end_page_num > start_page_num + 1 and number_of_words > 1000:
        term_type = "Topic"

    term_info = {
        "full_name": term_name,
        "name": name,
        "note": note,
        "alter_names": alter_names,
        "description": description,
        "starts_at": start_page_num,
        "ends_at": end_page_num,
        "number_of_words": number_of_words,
        "term_type": term_type,
        "position": position,
        "header": header,
        "reference_terms": reference_terms
    }
    return term_info

In [342]:
def parse_terms_info_from_txt(text, letters):
    lines = text.split("\n")
    previous_line = ""
    header = ""
    previous_description = ""
    previous_term_name = ""
    terms_info = []
    page_num = 0
    start_page_num = 1
    position = 1
    smallest_letter = letters[0][0]
    largest_letter = letters[1][0]

    for line in lines:
        #print(repr(line))
        line = line.strip()
        if line== "":
            continue

        if line.startswith("VOL."):
            continue

        header_pattern = re.compile(r'([A-Z][A-Z ]*)$|([\(\)\d\. ]+)$')
        header_match = header_pattern.match(line)
        if header_match:
            tmp_header = header_match.group().replace(" ", "")
            if largest_letter >= tmp_header[0] >= smallest_letter:
                # This might have outlier, e.g. header name starts with C, while last one was A, and the letters of the volume is A-D, Or these outliers are the result of OCR errors, should we consider them as wrong match??
                header = tmp_header
                page_num += 1
        else:
            word_pattern_str = "\p{Lu}[\p{Lu}\-\']*"
            word_ig_pattern_str = "\p{L}[\p{L}\-\']*"
            term_pattern_str = word_pattern_str +  "(\s\([\p{L}\p{N}\s\,\.]+\))?(,?\s"+ word_pattern_str +")*(\s(and)\s" + word_pattern_str + ")?(,?\sor\s" + word_pattern_str + ")?(\sof(\s"+  word_ig_pattern_str +")?)?[\,\.]"
            #term_pattern_str = word_pattern_str +  "(\s\([\p{L}\p{N}\s\,\.]+\))?(,?\s"+ word_pattern_str +")*(((\sand)|(,?\sor))(\s" + word_pattern_str + ")+)?(\sof(\s"+  word_ig_pattern_str +")*)?((\s"+  word_ig_pattern_str +")*)?[\,\.]"
            term_name_pattern = regex.compile(r"^" + term_pattern_str)
            term_name_match = term_name_pattern.match(line)

            if term_name_match:
                term_name = term_name_match.group()[:-1]
                has_consecutive_punctuations = "--" in term_name or "-'" in term_name or "'-" in term_name
                if largest_letter >= term_name[0] >= smallest_letter and not has_consecutive_punctuations:
                    # This might have outlier, e.g. term name starts with Z, while last one was A, and the letters of the volume is A-D
                    if previous_description == "" and (previous_term_name == term_name):
                        # previous description was empty, create new description based on this line
                        pass
                    else:
                        if previous_term_name == "":
                            # first term
                            pass
                        else:
                            end_page_num = page_num
                            term_info = create_term_info(end_page_num, start_page_num, previous_term_name, previous_description, position, header)
                            terms_info.append(term_info)
                            position += 1
                            if end_page_num > start_page_num:
                                position = 1
                    description = line[term_name_match.end():].strip()
                    previous_description = description
                    previous_term_name = term_name
                    start_page_num = page_num

                else:
                    overlap_count = find_overlap_count(previous_line, line)
                    if overlap_count > 3:
                        #print(f"previous line: {previous_line}, current line: {line}")
                        previous_description += line[overlap_count:]
                    else:
                        previous_description += " " + line

            else:
                overlap_count = find_overlap_count(previous_line, line)
                if overlap_count > 3:
                    #print(f"previous line: {previous_line}, current line: {line}")
                    previous_description += line[overlap_count:]
                else:
                    previous_description += " " + line
            previous_line = line

    end_page_num = page_num
    last_term_info = create_term_info(end_page_num, start_page_num, previous_term_name, previous_description, position, header)
    terms_info.append(last_term_info)

    return terms_info

In [343]:
def put_wrong_term_to_previous_one(terms_info, index):
    term_info = terms_info[index]
    terms_info[index - 1]["description"] += " " + term_info["full_name"] + ", " + term_info["description"]
    terms_info[index - 1]["number_of_words"] = len(terms_info[index - 1]["description"].split())
    terms_info[index - 1]["reference_terms"].extend(term_info["reference_terms"])
    terms_info[index - 1]["ends_at"] = term_info["ends_at"]
    if terms_info[index - 1]["term_type"] == "Article":
        if (len(terms_info[index - 1]["name"].split())) == 1 and terms_info[index - 1]["ends_at"] > terms_info[index - 1]["starts_at"] + 1 and terms_info[index - 1]["number_of_words"] > 1000:
            terms_info[index - 1]["term_type"] = "Topic"
    # remove wrongly matched term
    del terms_info[index]

In [344]:
def find_last_index_of(terms_info, term_name, r_index):
    while r_index < len(terms_info) and r_index >= 0:
        term_info = terms_info[r_index]
        if term_name == term_info["name"]:
            return r_index
        r_index -= 1

    return -1

In [345]:
 # Correct wrongly matched term, again, some outliers are the results of OCR errors, should we correct them??
def correct_terms(terms_info, letters):
    letter_parts = letters.split('-')
    min_letter = 'A'
    max_letter = 'Z'
    if len(letter_parts) == 1:
        min_letter = letters[0]
    elif len(letter_parts) == 2:
        min_letter = letter_parts[0][0]
        max_letter = letter_parts[1][0]
    print(f'min letter {min_letter}, max letter {max_letter}')
    current_letter = None
    index = 0
    while index < len(terms_info) - 1:
        term_info = terms_info[index]
        # Correct terms based on letter
        letter = term_info["name"][0]
        if letter < min_letter or letter > max_letter:
            put_wrong_term_to_previous_one(terms_info, index)
            continue
        if not current_letter:
            current_letter = letter
        else:
            if current_letter != letter:
                next_letter = terms_info[index + 1]['name'][0]
                if letter == next_letter:
                    # new letter
                    current_letter = letter
                else:
                    if current_letter == next_letter:
                        print(f"wrongly matched term: {term_info['name']}")
                        # wrongly matched term
                        # put the text back to previous term's description
                        put_wrong_term_to_previous_one(terms_info, index)
                        continue
        # correct terms whose name is combination of single letters with space, e.g. A A A
        term_name = term_info["name"]
        term_name_parts = term_name.split()
        if len(term_name_parts) > 1:
            is_single_letters = True
            for term_name_part in term_name_parts:
                if len(term_name_part) > 1:
                    is_single_letters = False

            if is_single_letters:
                print(term_name)
                put_wrong_term_to_previous_one(terms_info, index)
                continue

        # correct terms whose name appears before, but not continuous, e.g. AB, NAME, AB
        term_name = term_info["name"]
        last_index_term_name = find_last_index_of(terms_info, term_name, index -1)
        if last_index_term_name > -1 and last_index_term_name != index - 1:
            put_wrong_term_to_previous_one(terms_info, index)
            continue

        # correct terms with empty description
        if term_info["description"] == "" and len(term_info["alter_names"]) == 0:
            term_name = term_info["name"]
            if term_name == terms_info[index + 1]["name"][:len(term_name)]:
                del terms_info[index]
                continue
            if index > 0 and term_name == terms_info[index - 1]["name"][:len(term_name)]:
                del terms_info[index]
                continue
            if terms_info[index - 1]["description"][-1] != ".":
                terms_info[index - 1]["description"] += " " + term_name + "."
                terms_info[index - 1]["reference_terms"] = extract_reference_term_name(terms_info[index - 1]["description"])
                del terms_info[index]
                continue
        index += 1
    return terms_info

In [346]:
# load metadata
import pandas as pd
metadata_df = pd.read_json("../nls_metadata_dataframe", orient="index")

In [347]:
metadata_df

,editionNum,volumeNum,MMSID,editionTitle,editionSubTitle,editor,editor_date,genre,language,termsOfAddress,...,year,volumeId,permanentURL,publisherPersons,letters,part,supplementTitle,supplementSubTitle,supplementsTo,numberOfVolumes
0,1,1,992277653804341,"First edition, 1771, Volume 1, A-B",Illustrated with one hundred and sixty copperp...,"Smellie, William",1740-1795,encyclopedia,eng,None,...,1771,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",A-B,0,,,[],6
1,1,2,992277653804341,"First edition, 1771, Volume 2, C-L",Illustrated with one hundred and sixty copperp...,"Smellie, William",1740-1795,encyclopedia,eng,None,...,1771,144133902,https://digital.nls.uk/144133902,"[C. Macfarquhar, Colin Macfarquhar]",C-L,0,,,[],6
2,1,3,992277653804341,"First edition, 1771, Volume 3, M-Z",Illustrated with one hundred and sixty copperp...,"Smellie, William",1740-1795,encyclopedia,eng,None,...,1771,144133903,https://digital.nls.uk/144133903,"[C. Macfarquhar, Colin Macfarquhar]",M-Z,0,,,[],6
3,1,1,9929192893804340,"First edition, 1773, Volume 1, A-B",compiled upon a new plan. In which the differe...,"Smellie, William",1740-1795,encyclopedia,eng,None,...,1773,144850366,https://digital.nls.uk/144850366,[John Donaldson],A-B,0,,,[],6
4,1,2,9929192893804340,"First edition, 1773, Volume 2, C-L",compiled upon a new plan. In which the differe...,"Smellie, William",1740-1795,encyclopedia,eng,None,...,1773,144850367,https://digital.nls.uk/144850367,[John Donaldson],C-L,0,,,[],6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,8,12,9929777383804340,"Eighth edition, Volume 12, Hume-JOM",None,"Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,...,1853,193696087,https://digital.nls.uk/193696087,[],Hume-JOM,0,,,[],22
136,8,14,9929777383804340,"Eighth edition, Volume 14, Magnetism-MIH",None,"Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,...,1853,193696088,https://digital.nls.uk/193696088,[],Magnetism-MIH,0,,,[],22
137,8,17,9929777383804340,"Eighth edition, Volume 17, ORO-Plato",None,"Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,...,1853,193819046,https://digital.nls.uk/193819046,[],ORO-Plato,0,,,[],22
138,8,21,9929777383804340,"Eighth edition, Volume 21, T-ZWO",None,"Stewart, Dugald",1753-1828,encyclopedia,eng,Sir,...,1853,193819047,https://digital.nls.uk/193819047,[],T-ZWO,0,,,[],22


In [348]:
data_folder = "./data/"
all_info_list = []

edition_num = 1
year_published = 1771
numberOfTerms = None

for data_file in sorted(os.scandir(data_folder), key=lambda e: e.name):
    supported_file_patter = regex.compile(r"^EB_\d+_\d+_clean_cut\.txt$")
    data_file_name = data_file.name
    if supported_file_patter.match(data_file_name):
        print(f"Extracting data from {data_file_name} ......")
        data_file_name_parts = data_file_name.split("_")
        edition_num = int(data_file_name_parts[1])
        volume_num = int(data_file_name_parts[2])

        edition_volume_info = metadata_df[(metadata_df['editionNum'] == edition_num) & (metadata_df['year'] == year_published) & (metadata_df['volumeNum'] == volume_num)].iloc[0]
        # print(edition_volume_info)
        letters = edition_volume_info["letters"].split("-")
        text = open(data_file.path, "r").read()
        terms_info = parse_terms_info_from_txt(text, letters)

        # Correct wrongly matched term, again, some outliers are the results of OCR errors, should we correct them??
        terms_info = correct_terms(terms_info, edition_volume_info["letters"])

        print(f"number of terms in volume {volume_num}: {len(terms_info)}")
        print(f"page number of the last term of terms in volume {volume_num}: {terms_info[-1]['ends_at']}")

        for term_info in terms_info:
            all_info_list.append([term_info["name"], term_info["note"], term_info["alter_names"], term_info["reference_terms"],term_info["description"], term_info["starts_at"], term_info["ends_at"],term_info["position"],term_info["term_type"], data_file.path, term_info["number_of_words"], term_info["header"], edition_volume_info["letters"], edition_volume_info["part"], edition_volume_info["MMSID"],edition_volume_info["editionTitle"], edition_volume_info["editor"], edition_volume_info["editor_date"], edition_volume_info["genre"], edition_volume_info["language"], edition_volume_info["termsOfAddress"], edition_volume_info["numberOfPages"], edition_volume_info["physicalDescription"], edition_volume_info["place"], edition_volume_info["publisher"], edition_volume_info["referencedBy"], edition_volume_info["shelfLocator"], edition_volume_info["editionSubTitle"], edition_volume_info["volumeTitle"], edition_volume_info["year"], edition_volume_info["volumeId"], edition_volume_info["permanentURL"], edition_volume_info["publisherPersons"], edition_volume_info["volumeNum"], edition_volume_info["editionNum"], edition_volume_info["numberOfVolumes"],
                                  edition_volume_info["supplementTitle"], edition_volume_info["supplementSubTitle"], edition_volume_info["supplementsTo"], numberOfTerms])


# create pandas dataframe for information list
df = pd.DataFrame(all_info_list, columns=["term", "note", "alter_names","reference_terms", "definition",  "startsAt", "endsAt", "position", "termType", "filePath", "numberOfWords", "header","letters", "part", "MMSID", "editionTitle", "editor", "editor_date", "genre", "language", "termsOfAddress", "numberOfPages", "physicalDescription", "place", "publisher", "referencedBy", "shelfLocator", "editionSubTitle", "volumeTitle", "year", "volumeId", "permanentURL", "publisherPersons", "volumeNum", "editionNum", "numberOfVolumes", "supplementTitle", "supplementSubTitle", "supplementsTo", "numberOfTerms"])

Extracting data from EB_1_1_clean_cut.txt ......
min letter A, max letter B
wrongly matched term: BILLS
wrongly matched term: BOOK-KEEPING
wrongly matched term: BAHAMA
wrongly matched term: BORDURE
wrongly matched term: BRASS
wrongly matched term: A A
wrongly matched term: A A
wrongly matched term: A
A A A
wrongly matched term: B
A L
wrongly matched term: BOLIA
A A A
wrongly matched term: BB
A A
A A
wrongly matched term: BONES
wrongly matched term: BRACHLAEUS
wrongly matched term: BICEPS
A A
A A
wrongly matched term: B
wrongly matched term: A
wrongly matched term: B
wrongly matched term: BAL
wrongly matched term: ABILISS
wrongly matched term: BE LA ZOUCH
wrongly matched term: BSC CD, DS
wrongly matched term: BIL
wrongly matched term: B
A D
wrongly matched term: ALISTA
wrongly matched term: ANGUE
wrongly matched term: A
wrongly matched term: AGRICULTURE
wrongly matched term: ANAS
wrongly matched term: ARCHITECTURE
wrongly matched term: AEARDING OF MOOL
wrongly matched term: AENJAR
wrong

In [349]:
# Compare with Rosa's output
with open("output.json", 'r') as json_file:
    term_definition_related_pairs = json.load(json_file)

# Find terms do not start with 'A' or 'B'
term_count = 0
for term in term_definition_related_pairs:
    print(f"header: {term[0]}, page number: {term[1]}, term name: {term[2]}")
    # header, page_number, term, definition, related_terms
    if term[2][0] != 'A' and term[2][0] != 'B':
        term_count += 1
        #print(f"header: {term[0]}, page number: {term[1]}, term name: {term[2]}, definition: {term[3]}")

print(f"total number of terms: {len(term_definition_related_pairs)}")
print(f"number of terms which do not start with 'A' or 'B': {term_count}")
print(f"last page number: {term_definition_related_pairs[-1][1]}")

header: A B, page number: 1, term name: A,
header: A B, page number: 1, term name: AA
header: A B, page number: 1, term name: AABAM,
header: A B, page number: 1, term name: AACH,
header: A B, page number: 1, term name: AADE,
header: A B, page number: 1, term name: AAHUS,
header: A B, page number: 1, term name: AAM,
header: A B, page number: 1, term name: AAMA,
header: A B, page number: 1, term name: AARSEO,
header: A B, page number: 1, term name: AATTER,
header: A B, page number: 1, term name: AB,
header: A B, page number: 1, term name: AB
header: A B, page number: 1, term name: ASTRONOMY,
header: A B, page number: 1, term name: AB,
header: A B, page number: 1, term name: ABACATUAIA,
header: A B, page number: 1, term name: ABACAY,
header: A B, page number: 1, term name: ABACH,
header: A B, page number: 1, term name: ABACO,
header: A B, page number: 1, term name: ABACOA,
header: A B, page number: 1, term name: ABACOT,
header: A B, page number: 1, term name: ABACTORES,
header: A B, page 

In [350]:
df.to_json(r'./final_eb_1_dataframe_clean_Damon', orient="index")

In [9]:
df[df["term"] == "A---I"]

In [11]:
print(df.head(5))

,term,startsAt,endsAt
0,A,1,1
1,AABAM,1,1
2,AACH,1,1
3,AADE,1,1
4,AAHUS,1,1
...,...,...,...
15950,MBRCURY,975,975
15951,MEIAS,975,975
15952,SEM-METALS,975,976
15953,SEML-METAIS,976,976


In [20]:
df.loc[0]

In [25]:
# Statistic analysis of dataframe from this work
print("Number of terms: ", df.shape[0])
print("Number of topics: ", df[df["termType"] == "Topic"].shape[0])
print("Number of articles: ", df[df["termType"] == "Article"].shape[0])
print(df[df["year"] == 1773])

'6971124'

In [26]:
reference_terms_df = df[df['reference_terms'].apply(lambda x: len(x) > 0)]
print(reference_terms_df["reference_terms"].values.tolist())

{'TURGOT, ΑΝΝΕ ROBERT JACQUES, BARON': '7735483',
 'TURGOT, ΑΝΝΕ ROBERT': '6971124'}

In [1]:
# load the dataframe of EB first edition extracted using defoe
defoe_df_eb1 = pd.read_json('./final_eb_1_dataframe', orient="index")


In [8]:
defoe_df_eb1_1771 = defoe_df_eb1[defoe_df_eb1["year"] == 1771]

,term,note,alter_names,reference_terms,definition,startsAt,endsAt,position,termType,filePath,...,volumeId,permanentURL,publisherPersons,volumeNum,editionNum,numberOfVolumes,supplementTitle,supplementSubTitle,supplementsTo,numberOfTerms
62,ABBREVIATE OF ADJUDICATIONS,NaN,[],[SCOTS LAW],"in Scots law, an abstract or abridgement of a ...",5,5,5,Article,./data/EB_1_1_clean_cut.txt,...,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",1,1,6,,,[],NaN
213,ACADEMY OF ARCHITECTURE,NaN,[],[],established about the end of the year 1671 by ...,21,21,3,Article,./data/EB_1_1_clean_cut.txt,...,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",1,1,6,,,[],NaN
214,ACADEMY OF POLITICS,NaN,[],[ACAJOU],"is composed of six persons, who meet at the Lo...",21,23,4,Article,./data/EB_1_1_clean_cut.txt,...,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",1,1,6,,,[],NaN
325,ACCOUNT OF SALES,NaN,[],[BOOK-KEEPING],is an account given by one merchant to another...,31,31,58,Article,./data/EB_1_1_clean_cut.txt,...,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",1,1,6,,,[],NaN
340,ACCUMULATION OF DEGREES,NaN,[],[],"in an university, is the taking several of the...",34,34,11,Article,./data/EB_1_1_clean_cut.txt,...,144133901,https://digital.nls.uk/144133901,"[C. Macfarquhar, Colin Macfarquhar]",1,1,6,,,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15838,WOOD LOUSE,NaN,[],"[ONISCUS, PICUS]","in zoology. See ONISCUS. NOOD PECRER, in ornit...",959,959,23,Article,./data/EB_1_3_clean_cut.txt,...,144133903,https://digital.nls.uk/144133903,"[C. Macfarquhar, Colin Macfarquhar]",3,1,6,,,[],NaN
15840,WO F,NaN,[],[],"among manufacturers, the threads which the vea...",959,959,26,Article,./data/EB_1_3_clean_cut.txt,...,144133903,https://digital.nls.uk/144133903,"[C. Macfarquhar, Colin Macfarquhar]",3,1,6,,,[],NaN
15855,OTTON BASSET,NaN,[],[SURGERY],"a norough-town of Wil ishire, twenty five mmle...",960,960,12,Article,./data/EB_1_3_clean_cut.txt,...,144133903,https://digital.nls.uk/144133903,"[C. Macfarquhar, Colin Macfarquhar]",3,1,6,,,[],NaN
15910,"ZEA, INDIAN CORN",NaN,[],[],"in botany, a genus of the monoecitriandria cla...",967,967,5,Article,./data/EB_1_3_clean_cut.txt,...,144133903,https://digital.nls.uk/144133903,"[C. Macfarquhar, Colin Macfarquhar]",3,1,6,,,[],NaN


In [358]:
# Statistic analysis of dataframe from defoe
print("Number of terms: ", defoe_df_eb1_1771.shape[0])
print("Number of topics: ", defoe_df_eb1_1771[defoe_df_eb1_1771["typeTerm"] == "Topic"].shape[0])
print("Number of articles: ", defoe_df_eb1_1771[defoe_df_eb1_1771["typeTerm"] == "Article"].shape[0])

Number of terms:  8923
Number of topics:  64
Number of articles:  8859


In [359]:
# Find different between the topics from two dataframes
new_topics = df[df["termType"] == "Topic"]
defoe_df_eb1_1771_topics = defoe_df_eb1_1771[defoe_df_eb1_1771["typeTerm"] == "Topic"]
intersection = new_topics[new_topics["term"].isin(defoe_df_eb1_1771_topics["term"])]

print(f"{len(intersection)} topics have been recognised by both dataframe.")

# Remove the intersection from defoe's result
topic_defoe_df_eb1_1771_topics_dif_result = defoe_df_eb1_1771_topics[~defoe_df_eb1_1771_topics["term"].isin(intersection["term"])]

print(f"{len(topic_defoe_df_eb1_1771_topics_dif_result)} topics have only been recognised from defoe's result.")
# List the term names of topic in defoe's result only
print(topic_defoe_df_eb1_1771_topics_dif_result["term"].values.tolist())

16 topics have been recognised by both dataframe.
54 topics have only been recognised from defoe's result.
['AGRICULTURE', 'ALGEBRA', 'ANATOMY', 'ANNUITIES', 'ARCHITECTURE', 'ASTRONOMY', 'BARBASC', 'BOOKKEEPING', 'BOTANY', 'BREWING', 'BUTSBRIBRIEFS', 'BULBURWE', 'ISTKY', 'CHEMISTRY', 'COMMERCE', 'CONICSECTIONS', 'DUBDUEHWW', 'EXCS', 'EXCEXC', 'FARRIERY', 'FLUXIONS', 'FORTIFICATION', 'FUNFUL', 'GEOGRAPHY', 'OCXIPHY', 'GEOMETRY', 'GRAMMAR', 'HIVHOLHIVINGB', 'HORSEMANSHIP', 'LIW', 'TACIV', 'LOGIC', 'IITTV', 'MECHANICS', 'MIDWIFERY', 'MUSICK', 'MUTMRMUTILATION', 'NATURALHISTORY', 'NAVIGATION', 'OPTICS', 'PERSPECTIVE', 'PNEUMATICS', 'JIL', 'RELIGIONTHEOLOGY', 'SHORTHANDWRITING', 'SIMPLE', 'SUIIG', 'SURGEY', 'STANNING', 'ITASTAMI', 'TRICON', 'TRIGONOMETRY', 'VATVEIVAUDEMONT', 'WATCHCLOCKWOK']


In [360]:
# Remove the intersection from this work
topic_new_topics_dif_result = new_topics[~new_topics["term"].isin(intersection["term"])]

print(f"{len(topic_new_topics_dif_result)} topics have only been recognised from this work.")
# List the term names of topic in defoe's result only
print(topic_new_topics_dif_result["term"].values.tolist())

187 topics have only been recognised from this work.
['ABRIDGEMENT', 'ATHER', 'B', 'BB', 'ACRICULTURE', 'ACRICUETURE', 'ALCEBRA', 'ACIS', 'AI', 'AABLLIS', 'ATE-TETSLYST', 'AN', 'BIEL', 'BILLS', 'ANAS', 'BONES', 'ANOL', 'ANTITHENAR', 'ANTITHENAR', 'ACCESSORIUS', 'ATRABILARIAE', 'BBSL', 'APIS', 'APIPIODITA', 'ARANEA', 'ARDEA', 'ARISTOLUS', 'BEGIN', 'BESS', 'BLILS', 'AFYS', 'ACERIO', 'ARITHMETICE', 'BIS', 'ARK', 'ASTRO', 'AEINL', 'ASC', 'ALILBLISE', 'ABCDEEGH', 'ABCD', 'BEAUTY', 'BIBIO', 'BIPLES', 'BOOK-KEEPING', 'ALEILIS', 'ACBILIS', 'AILILILILSIS', 'BOTANIST', 'AC', 'BRIDEWELL', 'CANDISH', 'CANIS', 'CAPRA', 'CASTOR', 'CHEMISE', 'II', 'IF', 'DISTILLATION', 'ISTRY', 'LEAD', 'LXIV', 'IISTRY', 'ILES', 'FOU', 'CINCHONA', 'CLOTH', 'COBALT', 'COINAGE', 'COLUBER', 'COLUMBA', 'III', 'COMPANY', 'COMPARISON', 'CONGRUITY', 'CRITICISM', 'CROWN', 'DESIGN', 'DIAL', 'DE', 'LONG', 'DRAWING', 'DYEING', 'EDINBURGH', 'ICITY', 'ICITY', 'ENGLISH', 'ENGRAVING', 'EQUUS', 'EXCHANGE', 'IV', 'EHAMP', 'EXCISE', 'E

In [361]:
# Find different between the terms from two dataframes
intersection = df[df["term"].isin(defoe_df_eb1_1771["term"])]

print(f"{len(intersection)} terms have been recognised by both dataframe.")

# Remove the intersection from defoe's result
term_defoe_df_eb1_1771_result = defoe_df_eb1_1771[~defoe_df_eb1_1771["term"].isin(intersection["term"])]

# List the file path of term in defoe's result only
print(f"{len(term_defoe_df_eb1_1771_result)} terms have only been recognised by defoe's result.")

# Remove the intersection from df
term_dif_result = df[~df["term"].isin(intersection["term"])]

# List the file path of term in df only,
print(f"{len(term_dif_result)} terms have only been recognised by this work.")

6993 terms have been recognised by both dataframe.
2272 terms have only been recognised by defoe's result.
8962 terms have only been recognised by this work.


In [362]:
# Examine the terms only defoe's result
print(term_defoe_df_eb1_1771_result[["term", "definition"]][:50])
# Results shows that many fine term cannot be found in the txt file at all, we can do nothing about it.
# Some terms with format (All uppercase words with some all lowercase words) are partially recognised (only the uppercase's words), while our work can not identify them as terms. Need to improve!
# Some terms recognised here are not accurate, such as, ABU  SAN, an island on the coast of Africa, the term name should be "ABUSAN"

                  term                                         definition
0                   OR  A NEW A D I C T I A A, the name of several riv...
13                ABAC  AY, a barbarous name of a species of the pfitt...
27              ABACRE                   a town in the defarts of Arabia.
39             ABASCIA                the country of the Alcas. SCCALCAS.
40             ABAISED  Abaijfe, in heraldry, an epithet applied to ' ...
60          ABBEYBOYLE                    a town in the county of Rofcom-
62          ABBREVIATE  of adjudications, in Scots law, an abftradl or...
73              ABDEST  a term used for the legal purifications by wat...
74          ABDICARIAN  proposition, in logic, the same with a negativ...
80            ABELTREE  or ABELE-TREE, an obsolete name for a species ...
99             ABIDING  by a writing, in Scots law : When aperfoa soun...
101              PINUS                          of which it is a (pecies,
110          ABSORBENT  medicines, tef

In [363]:
# Examine the term names only in this work
print(term_dif_result["term"].values.tolist())

['AAMA', 'ABACAY', 'ABACOA', 'ABÆRE', 'ABAIEID', 'ABATOR', 'ABBEY-BOYLE', 'ABBREVIATE OF ADJUDICATIONS', 'ABBREVIATION', 'ABDESR', 'ABEL-TREE', 'ABELIANS', 'ABIGEATUS', 'ABIGIES', 'ABILITY', 'ABINGDON', 'AB-INTESTATE', 'ABISHERING', 'ABIT', 'ABJURATION', 'ABJURATION', 'ABJURATION', 'ABLAC', 'ABLACTATION', 'ABLACTATION', 'ABLACQUEATION', 'ABLATIVE', 'ABLAY', 'ABLECTI', 'ABLEGMINA', 'ABLET', 'ABLUENTS', 'ABLUTION', 'ABLUTION', 'ABO', 'ABOARD', 'ABOLITION', 'ABOLLA', 'ABOMASUS', 'ABOMINATION', 'ABORIGINES', 'ABORTION', 'ABORTION', 'ABORTIVE', 'ABOY', 'ABRA', 'ABRACADABRA', 'ABRAHAMITES', 'ABRAMIS', 'ABRENUNCIATION', 'ABROBANIA', 'ABROCHMENT', 'ABROGATION', 'ABROLKOS', 'ABRON', 'ABRONO', 'ABROTANOIDES', 'ABROTANOIDES', 'ABROTANUM', 'ABRUPTION', 'ABRUS', 'ABRUZZO', 'ABSCEDENTIA', 'ABSCESS', 'ABSCHARON', 'ABSCISSE', 'ABSCISSION', 'ABSCISSION', 'ABSCONSA', 'ABSENCE', 'ABSINTHIUM', 'ABSI', 'ABSOLUTE', 'ABSOLUTE', 'ABSOLUTION', 'ABSOLUTION', 'ABSOLUTION', 'ABSTRACTITIOUS', 'ABSYNTHIUM', 'ABUKES

In [364]:
# Examine the terms has more than three words
long_terms_df = df[df['term'].apply(lambda x: len(x.split()) > 3)]
print(long_terms_df["term"].values.tolist())
print(df.loc[9057])


['ANGULARIS, VULGO LEVATOR SCAPULAE PROPRIUS', 'AN OBJECT OF TASTE', 'ATHELING, ABELING, EDLING, ETHLING', 'BORGO DE VAL DE FARO', 'GENERAL PRINCIPLES OF GEOMETRY', 'GREEN MONRS AND NUNS', 'HIPPOCREPIS, COMMON HORSE-SHOE VETCH', 'CARAITES, CIRCUMCISION, LEVITES, PASSOVER', 'PENGUIN ISLAND AND BAT', 'PISSELKUM INDICUM, BARBADOES TAR', 'SANTA TE DE BAGOTA']
term                                                              GENEVA
note                                                                None
alter_names                                                           []
reference_terms                                                       []
definition             a city near the confines of France and Switzer...
startsAt                                                             793
endsAt                                                               794
position                                                               4
termType                                                  

In [365]:
# Examine the terms with empty description from this work
# case 1: ABESTON, a blundering way of writing Abestus. See \nABESTUS. (or See Parallels of \nALTITUDE.) ----- reference term of previous term
# case 2: AMAN, \nA M A \nAMAN, a port of Africa, in the kingdom of Morocco,apon the Atlantic --- footnote in a page, name of next term
# case 3: AGROS. \nA G Y \nAGROSTOGRAPHIA, signifies the history or description --- footnote in a page, partial name of next term
empty_terms_df = df[df['definition'].apply(lambda x: len(x) == 0)]
print(empty_terms_df)

                  term  note alter_names reference_terms definition  startsAt  \
1377            ALODES  None          []              []                  206   
2674           ABTEDES  None          []              []                  541   
4317             BREAD  None          []              []                  849   
4781           CABERAH  None          []              []                    9   
5333           CAUTERY  None          []              []                   87   
5462           CHAMÆCI  None          []              []                   99   
5670            CHORDS  None          []              []                  257   
6041             CIETY  None          []              []                  295   
6478         CORRECTOE  None          []              []                  385   
8997          JA NUARY  None          []              []                  763   
9005   KITCHEN- GARDEN  None          []              []                  776   
9219        GOOAE-NECE  None

In [366]:
# Examine the terms with empty description from this work
empty_defoe_df_eb1_1771_terms_df = defoe_df_eb1_1771[defoe_df_eb1_1771['definition'].apply(lambda x: len(x) == 0)]
print(empty_defoe_df_eb1_1771_terms_df)

Empty DataFrame
Columns: [MMSID, editionTitle, editor, editor_date, genre, language, termsOfAddress, numberOfPages, physicalDescription, place, publisher, referencedBy, shelfLocator, editionSubTitle, volumeTitle, year, volumeId, metsXML, permanentURL, publisherPersons, volumeNum, letters, part, editionNum, supplementTitle, supplementSubTitle, supplementsTo, numberOfVolumes, term, definition, relatedTerms, header, startsAt, endsAt, numberOfTerms, numberOfWords, positionPage, typeTerm, altoXML]
Index: []

[0 rows x 39 columns]


In [367]:
old_df = pd.read_json("final_eb_1_dataframe_old_clean_Damon", orient="index")

In [370]:
# Find different between the terms from two dataframes
intersection = df[df["term"].isin(old_df["term"])]

print(f"{len(intersection)} terms have been recognised by both dataframe.")

# Remove the intersection from defoe's result
term_old_df_eb1_1771_result = old_df[~old_df["term"].isin(intersection["term"])]

# List  term in old result only
print(f"{len(term_old_df_eb1_1771_result)} terms have only been recognised in old result.")

# Remove the intersection from df
term_dif_result = df[~df["term"].isin(intersection["term"])]

# List the file path of term in df only,
print(f"{len(term_dif_result)} terms have only been recognised by this work.")
print(term_dif_result["term"].values.tolist())

15938 terms have been recognised by both dataframe.
0 terms have only been recognised in old result.
17 terms have only been recognised by this work.
['ACQUA-CHE-TAVELLA', 'AIX-LA-CHAPELLE', 'ARNAY-LE-DUC', 'BERGEN-OP-ZOOM', 'BOURG-EN-BRESS', 'BOURG-SUR-MER', 'CANDLE-BERRY-TREE', 'CLOVE-JULY-TLOWER', 'COUNTER-SWALLOW-TAIL', 'CUCROW-SPIT-INSECT', 'DEAD-MENS-EVES', 'DEMI-SEMI-QUAVER', 'DEVIL-IN-A-BUSH', 'DISDIAPASON-SEMI-DIAPENTE', 'DISDIAPASON-SEMI-DITONE', 'GUN-SHOT-WOUNDS', 'ZV F Y']
